In [1]:
import pandas as pd
import numpy as np
import collections

In [2]:
num_boards = 10
price_csv_file_path = 'price_source_data.csv'
max_price = int(num_boards * 2000)

In [3]:
df = pd.read_csv(price_csv_file_path)
df = df[[col for col in df.columns if 'Price' in col or 'Board' in col]].copy()
df = df[~df.isnull().any(axis=1)]
df.columns = [item for sublist in [
    ['board_{}'.format(i), 'price_{}'.format(i)] for i in range(1, num_boards + 1)
    ] for item in sublist]
for col in [c for c in df if 'price' in c]:
    df[col] = df[col].str.slice(1).str.replace(",", ".").astype(float)
    df[col] = np.where(df[col] < num_boards*2, df[col]*1000, df[col]).astype(int)
df

,board_1,price_1,board_2,price_2,board_3,price_3,board_4,price_4,board_5,price_5,board_6,price_6,board_7,price_7,board_8,price_8,board_9,price_9,board_10,price_10
0,pepellou,2232,HampshireHawk,2580,pulsar512b,1768,rsmillie94,1848,BobAnderson52,1977,SteveMcKinnon,2173,ReheatedLeftovers,2011,cathode-ray-jepsen,2786,KapuBen,1503,DamianDragon,767
1,DJ-Logan,1207,LeoYee,2344,AACtrl,1983,primeideal,2028,TheScythian,1264,ericBG,1860,kamekura,1628,mahjoub98,2469,Pifisch,2322,slam267,2614
2,Razorneck,2244,Zubenelgenubi,2261,Karpyan,2342,N00b001,2268,kramopolis,1917,Wo_Cheng_Si_Le,2173,mfeeney88,2083,ipr,1966,vonSlattery,2766,Lexgrad,200
3,Michael-Westen,2847,bufferunderrun,1593,pioki,2294,kocimietka,2424,Roofies,3323,elTaljin,2448,mrlam,2047,Archilas,1942,diegohdragon,2263,epabarker,322
4,Like-a-hurricane,4239,Kjar,2368,Paulze2000,1732,AbuShaitan,1860,amanlikekennyken,1737,iamtheknight2,2293,Jputterg,1999,secretarisvogel,2207,ruhib,1363,Dawn4365,535
5,drchessdad,1354,mqj,1417,j123dh,2211,rodeo,2136,goirish,2085,TheGrandChessKnight,1824,SlimanX,1783,bungalowboi,1750,thiccclouds,2239,pat219,2291
6,FlokiTheCat,1965,j3084,1347,Seraya191,2103,weetbix2,1752,plastic_pusher,1677,STCLion,2389,vishAnan,1783,jameslatham,2338,jeremyjh,1965,Dingoe12,3821
7,rezoons,1595,y3LL3r,1534,Aphla,382,flaxl,2028,cowtone,1677,Gokuba,1812,housesounds,2203,psmathgeek,1894,AleksandrSudak,1727,matt_chess_play,2669
8,Aidoz,2668,f1nn33,2213,snorcal,1852,RunningInVienna,867,Aaronmb2,1570,pinbsg21,1884,marian_roman,1628,quaini,1417,King_killer27,1989,tpk1024,2757
9,jacade,2232,grabhispieces,1570,pie314271,2801,eigentor,2112,Narski,2025,Clarinetref,1968,flyhalf2k14,2023,Le-Penseur,2039,rhiehn,2286,Ezze_13,604


In [4]:
class Player:
    def __init__(self, name, price, board):
        self.name = name
        self.price = price
        self.board = board

In [5]:
# Set up "graph"
d = {}
for board in range(1, num_boards + 1):
    d[board] = collections.defaultdict(list)
    board_col = 'board_{}'.format(board)
    price_col = 'price_{}'.format(board)
    for ix, row in df[[board_col, price_col]].iterrows():
        # Rules to simplify problem
        if board == 1 or board == 10:
            if row[price_col] < 2600:
                continue
        elif board == 2 or board == 9:
            if row[price_col] < 2200:
                continue
        elif board == 3 or board == 8:
            if row[price_col] < 2100:
                continue
        else:
            if row[price_col] > 2000:
                continue
        d[board][row[price_col]].append(Player(row[board_col], row[price_col], board))

In [6]:
# Solve the problem
solutions = []
memo = collections.defaultdict(list)

def dfs(current_solution, board, price):
    # End condition
    if len(current_solution) == num_boards:
        if price == max_price:
            solutions.append(current_solution)
            csum = 0
            for b in range(num_boards - 1):
                csum += current_solution[b]
                memo[(b + 1, csum)].append(current_solution[b + 1:])
            return True
        return False
    
    # Use memo when possible
    if (board, price) in memo:
        for suffix in memo[(board, price)]:
            solutions.append(current_solution + suffix)
        return bool(memo[(board, price)])
    
    # Normal search
    result = False
    for new_price in d[board + 1]:
        sum_price = price + new_price
        if sum_price <= max_price:
            result = dfs(current_solution+[new_price], board + 1, price + new_price) or result
    
    # If search was unsuccessful, remember that
    if not result:
        memo[(board, price)] = []

    return result

dfs([], 0, 0)

True

In [7]:
# Get number of unique solutions
csum = 0
for i, solution in enumerate(solutions):
    unique = 1
    for j, price in enumerate(solution):
        unique *= len(d[j+1][price])
    csum += unique
csum

1222079

In [8]:
# Get random $20,000 team
random_team = np.random.randint(len(solutions))
print([(d[i + 1][price][0].name, d[i + 1][price][0].price/1000) 
       for i, price in enumerate(solutions[random_team])])

[('okoros', 3.185), ('Zubenelgenubi', 2.261), ('Seraya191', 2.103), ('RunningInVienna', 0.867), ('TheScythian', 1.264), ('Rolihlahla', 1.335), ('Rivimies', 1.747), ('Highonpotnuse', 2.302), ('Pifisch', 2.322), ('slam267', 2.614)]


In [9]:
# Get total number of possible teams
total = 1
for b in d:
    total *= len(d[b])
total

392074862400